In [1]:
import argparse
import json
import torch
import json
import os

In [2]:
def save_arguments(args, filepath):
    with open(filepath, 'w') as file:
        json.dump(vars(args), file)

def load_arguments(filepath):
    with open(filepath, 'r') as file:
        args_dict = json.load(file)
    return args_dict

# Optionally, repopulate argparse.ArgumentParser with these arguments
def repopulate_arguments(args_dict):
    parser = argparse.ArgumentParser(description="Example script")
    for key, value in args_dict.items():
        parser.add_argument(f'--{key}', type=type(value),default=value)
    return parser.parse_args([])

In [3]:
path = '../vqa_checkpoint/checkpoint_pretrain/llama2_7b_acc4_br5e3_correct_vnips'

loaded_args = load_arguments(path+'/args.json')

args = repopulate_arguments(loaded_args)
args.llama_model_path = '.' +args.llama_model_path
args.resume='../vqa_checkpoint/checkpoint_pretrain/llama2_7b_acc4_br5e3_correct_vnips/checkpoint_19.pth'

In [3]:
# path = '../vqa_checkpoint/checkpoint_pretrain/llama2_13b_acc8_br8e3_bs4_vnips'

# loaded_args = load_arguments(path+'/argsa.json')

# args = repopulate_arguments(loaded_args)
# args.llama_model_path = '.' +args.llama_model_path
# args.resume='../vqa_checkpoint/checkpoint_pretrain/llama2_13b_acc8_br8e3_bs4_vnips/checkpoint_18.pth'

In [4]:
import sys
sys.path.append('../')
from llama import Tokenizer
from llama_vqa import LLaMA_VQA
from dataloader import load_data

/home/users/nus/idmwyk/scratch/anaconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
checkpoint = torch.load(args.resume, map_location='cpu')
model.load_state_dict(checkpoint['model'], strict=False)
tokenizer = Tokenizer(model_path=f'{args.llama_model_path}./tokenizer.model')

In [7]:
def decoding(model, tokenizer, prompt1,prompt2,video=None):
    adapter = model.adapter_query.weight.reshape(-1, model.adapter_len, model.params.dim).unsqueeze(1)
    freqs= model.freqs_cis.cuda()
    
    tokens = [tokenizer.bos_id] + tokenizer.sp_model.encode(prompt1)
    query = torch.tensor(tokens, dtype=torch.int64).cuda()
    input_embedding = model.tok_embeddings(query)

    tokens_2 = tokenizer.sp_model.encode(prompt2)
    query_2 = torch.tensor(tokens_2, dtype=torch.int64).cuda()
    input_embedding_2 = model.tok_embeddings(query_2)
    tokens.extend(tokens_2)
    video = video.cuda().float()
    video/=video.norm(dim=-1,keepdim=True)
    if True:
        sim = video@model.memory.T

        sim = (sim*100).softmax(dim=-1)

        video = sim@model.memory
        video = video/video.norm(dim=-1,keepdim=True)
        
    video_feature = model.visual_proj(video)
    video_feature = (video_feature + model.temporal_emb.weight[:, :]).type(model.llamatype)
    vqa_video_start=input_embedding.shape[0]
    # print(video_feature.shape)
    input_embedding = torch.cat([input_embedding,video_feature,input_embedding_2])
    start_pos=0
    for j in range(10):
        vqa_h = input_embedding.unsqueeze(0)
        seqlen = vqa_h.shape[-2]
        freqs_cis = freqs[:seqlen]
        mask = None
        mask = torch.full((1, 1, seqlen, seqlen), float("-inf"), device=vqa_h.device)
        mask = torch.triu(mask, diagonal=0 + 1).type_as(vqa_h)

        for i, layer in enumerate(model.layers[-1 * model.adapter_layer:]):
            vqa_h = layer(vqa_h, start_pos, freqs_cis, mask, adapter[i].type(model.llamatype), vqa_video_start)
        vqa_h = model.norm(vqa_h)
        vqa_output = model.output(vqa_h)
        vqa_output = vqa_output.reshape(-1, model.vocab_size)
        vqa_output[-1,920]=-100
        vqa_output[-1,1128]=-100
        next_token = vqa_output[-1,:].argmax()
        tokens.append(next_token.item())
        token_emb = model.tok_embeddings(next_token.unsqueeze(0))
        input_embedding = torch.cat([input_embedding,token_emb],dim=0)
    return tokens

In [8]:
import os
dataset_path = '../data/videos/msrvtt/'
test_file = os.path.join(dataset_path,'test_videodatainfo.json')

with open(test_file,'r') as f:
    test_info = json.load(f)

In [9]:
annotations_gt = {}
annotations_gt['images'] = test_info['videos']

for sentence in test_info['sentences']:
    sentence['image_id'] = int(sentence['video_id'].replace('video',''))
    sentence['id'] = sentence['video_id']
annotations_gt['annotations'] =test_info['sentences']

In [10]:
import cv2
import numpy as np
from PIL import Image
def sample_images_from_video(video_path, num_samples=10):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    # Calculate total duration in seconds
    total_duration = total_frames / frame_rate
    # print(total_duration)
    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error opening video file.")
        return []

    # Calculate the interval for sampling
    interval = total_frames // num_samples

    # Initialize a list to store the sampled images
    sampled_images = []

    for i in range(num_samples):
        # Set the frame position
        frame_id = i * interval
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)

        # Read the frame
        ret, frame = cap.read()

        # If frame reading was successful, save the frame
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(frame)
            sampled_images.append(pil_image)
            
        else:
            print(f"Error reading frame at position {frame_id}")

    # Release the video capture object
    cap.release()

    return sampled_images, total_frames


In [11]:
import clip

In [12]:
clip_model, preprocess = clip.load("ViT-L/14")
clip_model.eval()
clip_model = clip_model.cuda()

In [13]:
from pycocotools.coco import COCO
from pycocoevalcap.eval import COCOEvalCap

from json import encoder
encoder.FLOAT_REPR = lambda o: format(o, '.3f')
import sys

In [14]:
annotation_file = os.path.join(dataset_path,'annotation_file')
with open(annotation_file,'w') as f:
    json.dump(annotations_gt,f)
coco = COCO(annotation_file)
video_ids = coco.getImgIds()

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [15]:
# prompt = "Instruction: Predict the answer based on the video and question.\nVideo:"
# prompt2 = "\nQuestion: Summarize the video.\nAnswer: It is a video showing"  #26.7

prompt = "Instruction: Predict the answer based on the video and question.\nVideo:"
prompt2 = "\nQuestion: Can you describe this video?\nAnswer: It is a video of"

In [ ]:
from tqdm import tqdm
results = []
    
for video_id in tqdm(video_ids[:]):
    with torch.no_grad():
        try:
            video_path = os.path.join(dataset_path,'TestVideo','video'+str(video_id)+'.mp4')
            sampled_images,_ = sample_images_from_video(video_path)

            image_features = [preprocess(image) for image in sampled_images]
            image_features = torch.stack(image_features,dim=0).cuda()
            image_features = clip_model.encode_image(image_features)
            image_features/=image_features.norm(dim=-1,keepdim=True)
            tokens = decoding(model,tokenizer,prompt,prompt2,image_features)
            generate_text = tokenizer.decode(tokens[:])
            generate_text = generate_text.split('It is a video of')[1].strip().split('.')[0]
            results.append({'image_id':video_id,'caption': generate_text})
        except:
            results.append({'image_id':video_id,'caption': 'A video'})

  5%|▌         | 151/2990 [02:28<36:09,  1.31it/s] 

In [ ]:
coco_result = coco.loadRes(results) 

coco_eval = COCOEvalCap(coco, coco_result)
coco_eval.evaluate()
# print output evaluation s|cores
scores = {}
for metric, score in coco_eval.eval.items():
    print(f"{metric}: {score:.3f}")
    scores[metric] = score
with open('13B_msrvtt_results.json','w') as f:
    json.dump(scores,f)